In [1]:
from games_setup import *
import SBMLLint.common.constants as cn
from SBMLLint.common.reaction import Reaction
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.som import SOM
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction
import collections
import tesbml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time
from scipy.linalg import lu, inv

Current Directory: /Users/woosubs/Desktop/ModelEngineering/SBMLLint/SBMLLint/notebook


In [2]:
# The following models are not loadable by simple SBML
EXCEPTIONS = ["BIOMD0000000075_url.xml",
              "BIOMD0000000081_url.xml",
              "BIOMD0000000094_url.xml",
              "BIOMD0000000353_url.xml",
              "BIOMD0000000596_url.xml",
             ]
data_dir=cn.BIOMODELS_DIR
# we can remove EXCEPTIONS from files, as they are not loaded by simpleSBML
raw_files = [f for f in os.listdir(data_dir) if f[:7] == "BIOMD00"]
files = [f for f in raw_files if f not in EXCEPTIONS]
paths = [os.path.join(data_dir, filename) for filename in files]

In [3]:
data_dir

'/Users/woosubs/Desktop/ModelEngineering/SBMLLint/SBMLLint/data/biomodels'

In [4]:
len(files)

651

In [5]:
# statistics columns
NUM_REACTIONS = "num_reactions(nonbdry)"
LP_ERROR = "lp_error"
GAMES_ERROR = "games_error"
GAMESPP_ERROR = "gamespp_error"
TYPEI_ERROR = "type1_error"
TYPEII_ERROR = "type2_error"
CANCELING_ERROR = "canceling_error"
ECHELON_ERROR = "echelon_error"
TYPEIII_ERROR = "type3_error"
result_columns = [NUM_REACTIONS,
                  LP_ERROR,
                  GAMES_ERROR,
                  GAMESPP_ERROR,
                  TYPEI_ERROR,
                  TYPEII_ERROR,
                  CANCELING_ERROR,
                  ECHELON_ERROR,
                  TYPEIII_ERROR]
## invertible matrix column? 
# INVERTIBLE = "l_inverse"

In [6]:
results = pd.DataFrame(0, index=files, columns=result_columns)
results[:5]

,num_reactions(nonbdry),lp_error,games_error,gamespp_error,type1_error,type2_error,canceling_error,echelon_error,type3_error
BIOMD0000000199_url.xml,0,0,0,0,0,0,0,0,0
BIOMD0000000189_url.xml,0,0,0,0,0,0,0,0,0
BIOMD0000000387_url.xml,0,0,0,0,0,0,0,0,0
BIOMD0000000397_url.xml,0,0,0,0,0,0,0,0,0
BIOMD0000000413_url.xml,0,0,0,0,0,0,0,0,0


In [7]:
simple = SimpleSBML()
simple.initialize(os.path.join(data_dir, "BIOMD0000000244_url.xml"))
s = StoichiometryMatrix(simple)
consistent = s.isConsistent()
print("consistent? ", consistent)

consistent?  False


//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_util.py:763: OptimizeWarning:

A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.



In [8]:
# LP only
simple = SimpleSBML()
count = 0
lp_start = time.time()
for file in files:
  count += 1
  if (count%100)==0:
    print("we are analyzing Model number:", count)
  try:
    simple.initialize(os.path.join(data_dir, file))
    s = StoichiometryMatrix(simple)
    num_reactions = s.stoichiometry_matrix.shape[1]
    results.at[file, NUM_REACTIONS] = num_reactions
    if num_reactions:
      consistent = s.isConsistent()
    else:
      consistent = -1
    results.at[file, LP_ERROR] = 1 - int(consistent)
  except:
    results.at[file, LP_ERROR] = -1
lp_end = time.time()
lp_time = lp_end - lp_start
print("Analysis finished!")
print("LP time:", lp_time)

/Users/woosubs/Desktop/ModelEngineering/SBMLLint/SBMLLint/SBMLLint/common/stoichiometry_matrix.py:88: OptimizeWarning:

Solving system with option 'cholesky':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'cholesky' to False.

/Users/woosubs/Desktop/ModelEngineering/SBMLLint/SBMLLint/SBMLLint/common/stoichiometry_matrix.py:88: OptimizeWarning:

Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=4.58452e-19): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rco

we are analyzing Model number: 100


//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=1.74587e-36): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=1.02314e-19): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=7.11744e-36): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=3.33065e-19): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=1.31373e-20): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:11

we are analyzing Model number: 200


//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=2.3643e-20): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=8.07424e-18): result may not be accurate.



...

we are analyzing Model number: 500


//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=4.40983e-19): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=3.67095e-36): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=1.19455e-19): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=1.10128e-35): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=3.27456e-20): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:11

we are analyzing Model number: 600


//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=3.53679e-19): result may not be accurate.



Analysis finished!
LP time: 44.31763792037964


In [9]:
lp_results = results[results[LP_ERROR] == 1]
len(lp_results)
print("(Mean) ISS for LP is:", np.mean(lp_results[NUM_REACTIONS]))
print("(STD) ISS for LP is:", np.std(lp_results[NUM_REACTIONS]))

(Mean) ISS for LP is: 55.51048951048951
(STD) ISS for LP is: 94.38982308045166


In [10]:
len(results[results[LP_ERROR]==1])

143

In [11]:
len(results[results[LP_ERROR]==-1])

5

In [12]:
# GAMES only
simple = SimpleSBML()
count = 0
games_start = time.time()
for file in files:
  count += 1
  if (count%100)==0:
    print("we are analyzing Model number:", count)
  try:
    simple.initialize(os.path.join(data_dir, file))
    m = GAMES_PP(simple)
    if simple.reactions:
      res = m.analyze(simple_games=True, error_details=False)
      results.at[file, GAMES_ERROR] = int(res)
      if res:
        gr = GAMESReport(m)
        summary = m.error_summary
        if m.type_one_errors:
          results.at[file, TYPEI_ERROR] = len(m.type_one_errors)
          report, error_num = gr.reportTypeOneError(m.type_one_errors, explain_details=True)
        if m.type_two_errors:
          results.at[file, TYPEII_ERROR] = len(m.type_two_errors)
          report, error_num = gr.reportTypeTwoError(m.type_two_errors, explain_details=True)
  except:
    results.at[file, GAMES_ERROR] = -1   
games_end = time.time()
games_time = games_end - games_start
print("Analysis finished!")
print("GAMES time:", games_time)

we are analyzing Model number: 100
we are analyzing Model number: 200
we are analyzing Model number: 300
we are analyzing Model number: 400
we are analyzing Model number: 500
we are analyzing Model number: 600
Analysis finished!
GAMES time: 107.20632076263428


In [13]:
print("number of detected errors: ", len(results[results[GAMES_ERROR]==1]))
print("number of GAMES but not in LP", len(results[(results[GAMES_ERROR]==1) & (results[LP_ERROR]!=1)]))

number of detected errors:  109
number of GAMES but not in LP 0


In [14]:
file

'BIOMD0000000298_url.xml'

In [15]:
results[results[GAMES_ERROR]==-1]

,num_reactions(nonbdry),lp_error,games_error,gamespp_error,type1_error,type2_error,canceling_error,echelon_error,type3_error
BIOMD0000000596_url.xml,0,-1,-1,0,0,0,0,0,0
BIOMD0000000081_url.xml,0,-1,-1,0,0,0,0,0,0
BIOMD0000000075_url.xml,0,-1,-1,0,0,0,0,0,0
BIOMD0000000094_url.xml,0,-1,-1,0,0,0,0,0,0
BIOMD0000000353_url.xml,0,-1,-1,0,0,0,0,0,0


In [24]:
# GAMES+
# file, GAMES_ERROR coding:
# 0; normal - no error found
# -1; not loaded or error found
# 1; normal - error found
# 2; echelon error found, but it is not explainable
# 3; type III error found, but it is not explainable
simple = SimpleSBML()
count = 0
gamespp_start = time.time()
for file in files:
  count += 1
  if (count%100)==0:
    print("we are analyzing Model number:", count)
  try:
    simple.initialize(os.path.join(data_dir, file))
    m = GAMES_PP(simple)
    if simple.reactions:
      res = m.analyze(simple_games=False, error_details=False)
      results.at[file, GAMESPP_ERROR] = int(res)
      if res:
        if m.echelon_errors or m.type_three_errors:
          try:
            #k = inv(m.lower)
            k = np.linalg.inv(m.lower)
          except:
            print("model %s has as a singular L matrix:" % file)
        condition_number = np.linalg.cond(m.lower)
#         if condition_number > 300:
#           print("*****The L matrix of the model %s has a condition number %f*****" % (file, condition_number))
        gr = GAMESReport(m)
        summary = m.error_summary
        if m.type_one_errors:
          results.at[file, TYPEI_ERROR] = len(m.type_one_errors)
          report, error_num = gr.reportTypeOneError(m.type_one_errors, explain_details=True)
        if m.type_two_errors:
          results.at[file, TYPEII_ERROR] = len(m.type_two_errors)
          report, error_num = gr.reportTypeTwoError(m.type_two_errors, explain_details=True)
        if m.canceling_errors:
          results.at[file, CANCELING_ERROR] = len(m.canceling_errors)
          report, error_num = gr.reportCancelingError(m.canceling_errors, explain_details=True)
        if m.echelon_errors:
          #print("Model %s has an echelon error:" % file)
          results.at[file, ECHELON_ERROR] = len(m.echelon_errors)
          report, error_num = gr.reportEchelonError(m.echelon_errors, explain_details=True)
          if report is False:
            results.at[file, GAMESPP_ERROR] = 2
            print("Model %s has an inexplainable Echelon Error" % file)
            print("As the lower matrix has a condition number %f" % condition_number)
            print("Decide if the matrix is invertible")
        if m.type_three_errors:
          #print("Model %s has a type III error:" % file)
          results.at[file, TYPEIII_ERROR] = len(m.type_three_errors)
          report, error_num = gr.reportTypeThreeError(m.type_three_errors, explain_details=True)
          if report is False:
            results.at[file, GAMESPP_ERROR] = 3
            print("Model %s has an inexplainable Type III Error" % file)
            print("As the lower matrix has a condition number %f" % condition_number)
            print("Decide if the matrix is invertible")
  except:
    results.at[file, GAMES_ERROR] = -1   
gamespp_end = time.time()
gamespp_time = gamespp_end - gamespp_start
print("\nAnalysis finished!")
print("GAMES++ time:", gamespp_time)

we are analyzing Model number: 100
Model BIOMD0000000105_url.xml has an inexplainable Echelon Error
As the lower matrix has a condition number 27.862143
Decide if the matrix is invertible
we are analyzing Model number: 200
Model BIOMD0000000245_url.xml has an inexplainable Echelon Error
As the lower matrix has a condition number 6.150046
Decide if the matrix is invertible
Model BIOMD0000000497_url.xml has an inexplainable Echelon Error
As the lower matrix has a condition number 128.983818
Decide if the matrix is invertible
we are analyzing Model number: 300
we are analyzing Model number: 400
we are analyzing Model number: 500
we are analyzing Model number: 600
Model BIOMD0000000243_url.xml has an inexplainable Type III Error
As the lower matrix has a condition number 7.533800
Decide if the matrix is invertible

Analysis finished!
GAMES++ time: 188.87111496925354


In [21]:
print("number of detected errors: ", len(results[results[GAMESPP_ERROR]==1]))
print("number of GAMES errors not in LP", len(results[(results[GAMESPP_ERROR]==1) & (results[LP_ERROR]!=1)]))
len(results[results[GAMESPP_ERROR]==-1])

number of detected errors:  133
number of GAMES errors not in LP 0


0

In [22]:
len(results[results[GAMESPP_ERROR]==2])

3

In [23]:
len(results[results[GAMESPP_ERROR]==3])

1

In [20]:
results[results[GAMESPP_ERROR]==3]

,num_reactions(nonbdry),lp_error,games_error,gamespp_error,type1_error,type2_error,canceling_error,echelon_error,type3_error
BIOMD0000000628_url.xml,584,0,0,3,0,0,0,0,3


In [ ]:
simple = load_file_from_games(574)
m = GAMES_PP(simple)
res = m.analyze(simple_games=False, error_details=True)

In [71]:
m.lower

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [72]:
np.linalg.det(m.lower)

1.0

In [76]:
m.lower_inverse

,rbp_binding_to_cam_TR_0_0,rbp_binding_to_cam_TR_ACD_0,tbp_binding_to_cam_RT_AD_0,rbp_binding_to_cam_TT_B_0,tbp_binding_to_cam_RR_ABD_0,tbp_binding_to_cam_RT_CD_0,tbp_binding_to_cam_RT_B_0,tbp_binding_to_cam_RT_C_0,tbp_binding_to_cam_RR_ACD_0,rbp_binding_to_cam_RT_BD_0,...,ca_binding_to_cam_RR_BCD_0_on_site_A,ca_binding_to_cam_RT_0_tbp_on_site_A,ca_binding_to_cam_TT_ACD_rbp_on_site_B,ca_binding_to_cam_TT_ABD_rbp_on_site_C,ca_binding_to_cam_TT_ABC_rbp_on_site_D,ca_binding_to_cam_RR_ABD_0_on_site_C,ca_binding_to_cam_TT_BCD_tbp_on_site_A,ca_binding_to_cam_TT_ACD_tbp_on_site_B,ca_binding_to_cam_TT_ABD_tbp_on_site_C,ca_binding_to_cam_TT_ABC_tbp_on_site_D
rbp_binding_to_cam_TR_0_0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rbp_binding_to_cam_TR_ACD_0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tbp_binding_to_cam_RT_AD_0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rbp_binding_to_cam_TT_B_0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tbp_binding_to_cam_RR_ABD_0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tbp_binding_to_cam_RT_CD_0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tbp_binding_to_cam_RT_B_0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tbp_binding_to_cam_RT_C_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tbp_binding_to_cam_RR_ACD_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rbp_binding_to_cam_RT_BD_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
simple.getReaction("rbp_binding_to_cam_RR_0_0")

rbp_binding_to_cam_RR_0_0: rbp + cam_RR_0_0 -> cam_RR_0_rbp; cytosol * (kon_rbp * rbp * cam_RR_0_0 - koff_rbp_RR * cam_RR_0_rbp)

In [95]:
op_series[pd.Series.to_numpy(op_series).nonzero()[0]]

{ca}   -3.0
Name: rbp_binding_to_cam_RR_0_0, dtype: float64

In [104]:
op_mat = gr.getOperationMatrix()
print(op_mat["rbp_binding_to_cam_RR_0_0"])

rbp_binding_to_cam_TR_0_0                 4.503600e+15
rbp_binding_to_cam_TR_ACD_0              -2.251800e+15
tbp_binding_to_cam_RT_AD_0                2.251800e+15
rbp_binding_to_cam_TT_B_0                 2.251800e+15
tbp_binding_to_cam_RR_ABD_0               0.000000e+00
                                              ...     
ca_binding_to_cam_RR_ABD_0_on_site_C      0.000000e+00
ca_binding_to_cam_TT_BCD_tbp_on_site_A    5.952029e-16
ca_binding_to_cam_TT_ACD_tbp_on_site_B    5.952029e-16
ca_binding_to_cam_TT_ABD_tbp_on_site_C    0.000000e+00
ca_binding_to_cam_TT_ABC_tbp_on_site_D    0.000000e+00
Name: rbp_binding_to_cam_RR_0_0, Length: 512, dtype: float64


In [111]:
m.rref_operation.dot(m.lower_inverse)["rbp_binding_to_cam_RR_0_0"]

rbp_binding_to_cam_TR_0_0                 4.503600e+15
rbp_binding_to_cam_TR_ACD_0              -2.251800e+15
tbp_binding_to_cam_RT_AD_0                2.251800e+15
rbp_binding_to_cam_TT_B_0                 2.251800e+15
tbp_binding_to_cam_RR_ABD_0               0.000000e+00
                                              ...     
ca_binding_to_cam_RR_ABD_0_on_site_C      0.000000e+00
ca_binding_to_cam_TT_BCD_tbp_on_site_A    5.952029e-16
ca_binding_to_cam_TT_ACD_tbp_on_site_B    5.952029e-16
ca_binding_to_cam_TT_ABD_tbp_on_site_C    0.000000e+00
ca_binding_to_cam_TT_ABC_tbp_on_site_D    0.000000e+00
Name: rbp_binding_to_cam_RR_0_0, Length: 512, dtype: float64

In [122]:
m.lower_inverse.iloc[4]

rbp_binding_to_cam_TR_0_0                 0.0
rbp_binding_to_cam_TR_ACD_0               0.0
tbp_binding_to_cam_RT_AD_0                0.0
rbp_binding_to_cam_TT_B_0                 0.0
tbp_binding_to_cam_RR_ABD_0               1.0
                                         ... 
ca_binding_to_cam_RR_ABD_0_on_site_C      0.0
ca_binding_to_cam_TT_BCD_tbp_on_site_A    0.0
ca_binding_to_cam_TT_ACD_tbp_on_site_B    0.0
ca_binding_to_cam_TT_ABD_tbp_on_site_C    0.0
ca_binding_to_cam_TT_ABC_tbp_on_site_D    0.0
Name: tbp_binding_to_cam_RR_ABD_0, Length: 512, dtype: float64

In [105]:
np.linalg.det(op_mat)

1.2340589426006188

In [169]:
TOLERANCE

0.0001

In [200]:
sum((abs(rref_df) > 1/TOLERANCE).sum())

0

In [171]:
echelon_df = np.round(m.echelon_df, 3)
rref_operation = np.identity(echelon_df.T.shape[0])
rref_operation = pd.DataFrame(rref_operation,
                     index = echelon_df.columns,
                     columns = echelon_df.columns)
# now update the operation matrix, finally multiply (dot product) two matrices
for idx, colname in enumerate(echelon_df.columns):
  reaction_series = echelon_df[colname]
  # Find the first nonzero values
  # Deprecation: instead of np.nonzero(Series), use Series.to_numpy().nonzero()
  # to fix error on GitHub - not use to_numpy here
  ##nonzero_idx = echelon_df[colname].to_numpy().nonzero()[0]
  nonzero_idx = np.array([idx for idx, val in enumerate(echelon_df[colname]) if val != 0])
  # Skip if there is no nonzero value or if it is first reaction
  if not nonzero_idx.any() or idx == 0:
    continue
  nonzero_species = reaction_series.index[nonzero_idx[0]]
  nonzero_value = reaction_series[nonzero_idx[0]]
  if abs(nonzero_value) > 1/TOLERANCE or abs(nonzero_value) < TOLERANCE:
    print("We skip the nonzero value: %f" % nonzero_value)
    continue
  # figure out if the nonzero_value is out of range (too big or too small)
  if (abs(nonzero_value) > 1/TOLERANCE) or abs(nonzero_value) < TOLERANCE:
    print("index: %d and colname: %s, nonzero value is abnormal as: %f" % (idx, colname, nonzero_value))
    print(nonzero_value == 0.0)
  # find current nonzero index
  current_echelon_t = rref_operation.dot(echelon_df.T).T
  for prev_colname in current_echelon_t.columns[:idx]:
    if np.round(current_echelon_t[prev_colname][nonzero_species], 3) != 0.0:
      reduction_value = current_echelon_t[prev_colname][nonzero_species]
      rref_operation.at[prev_colname, colname] = (-1.0) * reduction_value / nonzero_value
  rref_df = np.round(rref_operation.dot(echelon_df.T).T, 3)

In [186]:
k = np.round(m.echelon_df, 3)
k < TOLERANCE

,rbp_binding_to_cam_TR_0_0,rbp_binding_to_cam_TR_ACD_0,tbp_binding_to_cam_RT_AD_0,rbp_binding_to_cam_TT_B_0,tbp_binding_to_cam_RR_ABD_0,tbp_binding_to_cam_RT_CD_0,tbp_binding_to_cam_RT_B_0,tbp_binding_to_cam_RT_C_0,tbp_binding_to_cam_RR_ACD_0,rbp_binding_to_cam_RT_BD_0,...,ca_binding_to_cam_RR_BCD_0_on_site_A,ca_binding_to_cam_RT_0_tbp_on_site_A,ca_binding_to_cam_TT_ACD_rbp_on_site_B,ca_binding_to_cam_TT_ABD_rbp_on_site_C,ca_binding_to_cam_TT_ABC_rbp_on_site_D,ca_binding_to_cam_RR_ABD_0_on_site_C,ca_binding_to_cam_TT_BCD_tbp_on_site_A,ca_binding_to_cam_TT_ACD_tbp_on_site_B,ca_binding_to_cam_TT_ABD_tbp_on_site_C,ca_binding_to_cam_TT_ABC_tbp_on_site_D
{cam_TR_0_rbp},False,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
{cam_TR_ACD_rbp},True,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
{cam_RT_AD_tbp},True,True,False,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
{cam_TT_B_rbp},True,True,True,False,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
{cam_RR_ABD_tbp},True,True,True,True,False,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
{cam_RT_CD_tbp},True,True,True,True,True,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
{cam_RT_B_tbp},True,True,True,True,True,True,False,True,True,True,...,True,True,True,True,True,True,True,True,True,True
{cam_RT_C_tbp},True,True,True,True,True,True,True,False,True,True,...,True,True,True,True,True,True,True,True,True,True
{cam_RR_ACD_tbp},True,True,True,True,True,True,True,True,False,True,...,True,True,True,True,True,True,True,True,True,True
{cam_RT_BD_rbp},True,True,True,True,True,True,True,True,True,False,...,True,True,True,True,True,True,True,True,True,True


In [152]:
rref_operation

,rbp_binding_to_cam_TR_0_0,rbp_binding_to_cam_TR_ACD_0,tbp_binding_to_cam_RT_AD_0,rbp_binding_to_cam_TT_B_0,tbp_binding_to_cam_RR_ABD_0,tbp_binding_to_cam_RT_CD_0,tbp_binding_to_cam_RT_B_0,tbp_binding_to_cam_RT_C_0,tbp_binding_to_cam_RR_ACD_0,rbp_binding_to_cam_RT_BD_0,...,ca_binding_to_cam_RR_BCD_0_on_site_A,ca_binding_to_cam_RT_0_tbp_on_site_A,ca_binding_to_cam_TT_ACD_rbp_on_site_B,ca_binding_to_cam_TT_ABD_rbp_on_site_C,ca_binding_to_cam_TT_ABC_rbp_on_site_D,ca_binding_to_cam_RR_ABD_0_on_site_C,ca_binding_to_cam_TT_BCD_tbp_on_site_A,ca_binding_to_cam_TT_ACD_tbp_on_site_B,ca_binding_to_cam_TT_ABD_tbp_on_site_C,ca_binding_to_cam_TT_ABC_tbp_on_site_D
rbp_binding_to_cam_TR_0_0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rbp_binding_to_cam_TR_ACD_0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tbp_binding_to_cam_RT_AD_0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rbp_binding_to_cam_TT_B_0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tbp_binding_to_cam_RR_ABD_0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tbp_binding_to_cam_RT_CD_0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tbp_binding_to_cam_RT_B_0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tbp_binding_to_cam_RT_C_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tbp_binding_to_cam_RR_ACD_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rbp_binding_to_cam_RT_BD_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [98]:
op_series = m.rref_df["rbp_binding_to_cam_RR_0_0"]

reaction_op = gr.convertOperationSeriesToReactionOperations(op_series)
print(reaction_op)
print(m.convertReactionToSOMReaction(simple.getReaction("rbp_binding_to_cam_RR_0_0")))

[ReactionOperation(reaction='{ca}', operation=-3.0)]
rbp_binding_to_cam_RR_0_0: {rbp} + {cam_RR_0_0=cam_RT_0_0=cam_TR_0_0=cam_TT_0_0} -> {cam_RR_0_rbp}


In [102]:
op_series

{cam_TR_0_rbp}                                               0.0
{cam_TR_ACD_rbp}                                             0.0
{cam_RT_AD_tbp}                                              0.0
{cam_TT_B_rbp}                                               0.0
{cam_RR_ABD_tbp}                                             0.0
                                                            ... 
{cam_RR_ABC_0=cam_RT_ABC_0=cam_TR_ABC_0=cam_TT_ABC_0}        0.0
{cam_RR_ABD_0=cam_RT_ABD_0=cam_TR_ABD_0=cam_TT_ABD_0}        0.0
{cam_RR_ACD_0=cam_RT_ACD_0=cam_TR_ACD_0=cam_TT_ACD_0}        0.0
{cam_RR_BCD_0=cam_RT_BCD_0=cam_TR_BCD_0=cam_TT_BCD_0}       -0.0
{cam_RR_ABCD_0=cam_RT_ABCD_0=cam_TR_ABCD_0=cam_TT_ABCD_0}    0.0
Name: rbp_binding_to_cam_RR_0_0, Length: 147, dtype: float64

In [101]:
simple.getReaction(reaction_op[0].reaction)

In [86]:
m.rref_df

,{cam_TR_0_rbp},{cam_TR_ACD_rbp},{cam_RT_AD_tbp},{cam_TT_B_rbp},{cam_RR_ABD_tbp},{cam_RT_CD_tbp},{cam_RT_B_tbp},{cam_RT_C_tbp},{cam_RR_ACD_tbp},{cam_RT_BD_rbp},...,{cam_RR_AC_0=cam_RT_AC_0=cam_TR_AC_0=cam_TT_AC_0},{cam_RR_AD_0=cam_RT_AD_0=cam_TR_AD_0=cam_TT_AD_0},{cam_RR_BC_0=cam_RT_BC_0=cam_TR_BC_0=cam_TT_BC_0},{cam_RR_BD_0=cam_RT_BD_0=cam_TR_BD_0=cam_TT_BD_0},{cam_RR_CD_0=cam_RT_CD_0=cam_TR_CD_0=cam_TT_CD_0},{cam_RR_ABC_0=cam_RT_ABC_0=cam_TR_ABC_0=cam_TT_ABC_0},{cam_RR_ABD_0=cam_RT_ABD_0=cam_TR_ABD_0=cam_TT_ABD_0},{cam_RR_ACD_0=cam_RT_ACD_0=cam_TR_ACD_0=cam_TT_ACD_0},{cam_RR_BCD_0=cam_RT_BCD_0=cam_TR_BCD_0=cam_TT_BCD_0},{cam_RR_ABCD_0=cam_RT_ABCD_0=cam_TR_ABCD_0=cam_TT_ABCD_0}
rbp_binding_to_cam_TR_0_0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0
rbp_binding_to_cam_TR_ACD_0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0
tbp_binding_to_cam_RT_AD_0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0
rbp_binding_to_cam_TT_B_0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0
tbp_binding_to_cam_RR_ABD_0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tbp_binding_to_cam_RT_CD_0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0
tbp_binding_to_cam_RT_B_0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0
tbp_binding_to_cam_RT_C_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0
tbp_binding_to_cam_RR_ACD_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,0.0
rbp_binding_to_cam_RT_BD_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,-0.0


In [84]:
m.echelon_errors[0]

rbp_binding_to_cam_RR_0_0: 3.00 {ca} -> 

In [70]:
gr = GAMESReport(m)
echelon_report, error_num = gr.reportEchelonError([m.echelon_errors[0]], explain_details=True)
print(echelon_report)
print(error_num)

False
[33]


In [55]:
np.linalg.det(np.linalg.inv(m.lower))

1.0

In [37]:
# # GAMES++ 
# simple = SimpleSBML()
# count = 0
# gamespp_start = time.time()
# for file in files[82:89]:
#   print("We are doing file", file)
#   count += 1
#   if (count%100)==0:
#     print("we are analyzing Model number:", count)
#   simple.initialize(os.path.join(data_dir, file))
#   m = GAMES_PP(simple)
#   if simple.reactions:
#     res = m.analyze(simple_games=False, error_details=False)
#     results.at[file, GAMESPP_ERROR] = int(res)
#     if res:
#       gr = GAMESReport(m)
#       summary = m.error_summary
#       if m.type_one_errors:
#         results.at[file, TYPEI_ERROR] = len(m.type_one_errors)
#         report, error_num = gr.reportTypeOneError(m.type_one_errors)
#       if m.type_two_errors:
#         results.at[file, TYPEII_ERROR] = len(m.type_two_errors)
#         report, error_num = gr.reportTypeTwoError(m.type_two_errors)
#       if m.canceling_errors:
#         results.at[file, CANCELING_ERROR] = len(m.canceling_errors)
#         report, error_num = gr.reportCancelingError(m.canceling_errors)
#       if m.echelon_errors:
#         results.at[file, ECHELON_ERROR] = len(m.echelon_errors)
#         report, error_num = gr.reportEchelonError(m.echelon_errors)
#       if m.type_three_errors:
#         results.at[file, TYPEIII_ERROR] = len(m.type_three_errors)
#         report, error_num = gr.reportTypeThreeError(m.type_three_errors)

We are doing file BIOMD0000000032_url.xml
We are doing file BIOMD0000000022_url.xml
We are doing file BIOMD0000000479_url.xml
We are doing file BIOMD0000000501_url.xml
We are doing file BIOMD0000000469_url.xml
We are doing file BIOMD0000000511_url.xml
We are doing file BIOMD0000000354_url.xml


In [28]:
results.loc["BIOMD0000000479_url.xml", LP_ERROR]

1

In [23]:
results.head()

,num_reactions(nonbdry),lp_error,games_error,gamespp_error,type1_error,type2_error,canceling_error,echelon_error,type3_error
BIOMD0000000199_url.xml,10,0,0,0,0,0,0,0,0
BIOMD0000000189_url.xml,13,1,0,1,4,0,0,0,0
BIOMD0000000387_url.xml,4,0,0,0,0,0,0,0,0
BIOMD0000000397_url.xml,29,0,0,0,0,0,0,0,0
BIOMD0000000413_url.xml,5,1,0,1,1,0,0,0,0


In [35]:
import networkx as nx
som_example = list(m.nodes)[-1]
PathNodesReactions = collections.namedtuple('PathNodesReactions',
    'node1 node2 reactions')
def getSOMPath(som, mole1, mole2):
  """
  Create an undirected graph between
  two molecules within a SOM
  and find the shortest path
  :param SOM som:
  :param str mole1:
  :param str mole2:
  :return PathNodesReactions result_data:
  """   
  molecule1 = simple.getMolecule(mole1).name
  moelcule2 = simple.getMolecule(mole2).name
  # construct undirected graph
  subg = nx.Graph()
  # here, every reaction is 1-1 reaction
  for reaction in list(som.reactions):
    print(reaction.makeIdentifier(is_include_kinetics=False))
    node1 = reaction.reactants[0].molecule.name
    node2 = reaction.products[0].molecule.name
    if subg.has_edge(node1, node2):
      reaction_label = subg.get_edge_data(node1, node2)[cn.REACTION]
      # if reaction.label is not already included in the attribute,
      if reaction.label not in set(reaction_label):
        reaction_label = reaction_label + [reaction.label]
    else:
      reaction_label = [reaction.label]    
    subg.add_edge(node1, node2, reaction=reaction_label)
  print(list(subg.edges))
  path = [p for p in nx.shortest_path(subg, 
                                      source=mole1, 
                                      target=mole2)]
  # if result has more than 1 element need a for loop
  print("We found the shortest path from " + mole1 + " to " + mole2)
  result_data = []
  for idx in range(len(path)-1):
    print(path[idx] + "=" + path[idx+1], end=" ")
    edge_reactions = subg.get_edge_data(path[idx], path[idx+1])[cn.REACTION]
    print("by reaction(s)", edge_reactions)
    result_data.append(PathNodesReactions(node1=path[idx], 
                                          node2=path[idx+1],
                                          reactions=edge_reactions))
  return result_data

In [131]:
som_path = getSOMPath(som_example, 's260', 's270')
type(som_path)

r58: s36 -> s232
re65: s260 -> s232
re64: s270 -> s232
[('s36', 's232'), ('s232', 's260'), ('s232', 's270')]
We found the shortest path from s260 to s270
s260=s232 by reaction(s) ['re65']
s232=s270 by reaction(s) ['re64']


list

In [134]:
for pat in som_path:

  print(pat.node1 + " = " + pat.node2 + " by", end=" ")
  for r in pat.reactions:
    reaction = simple.getReaction(r)
    print(reaction.makeIdentifier(is_include_kinetics=False))

s260 = s232 by re65: s260 -> s232
s232 = s270 by re64: s270 -> s232


In [124]:
print(subg.edges)
print(subg.has_edge('s260', 's232'))
print(subg.has_edge('s232', 's260'))

[('s36', 's232'), ('s232', 's260'), ('s232', 's270')]
True
True


In [87]:
som = som_example
mole1 = 's260'
mole2 = 's270'
molecule1 = simple.getMolecule(mole1).name
moelcule2 = simple.getMolecule(mole2).name
# construct undirected graph
subg = nx.Graph()
# subg.add_nodes_from(som.molecules)
# here, every reaction is 1-1 reaction
for reaction in list(som.reactions):
  print(reaction.makeIdentifier(is_include_kinetics=False))
  node1 = reaction.reactants[0].molecule.name
  node2 = reaction.products[0].molecule.name
  if subg.has_edge(node1, node2):
    reaction_label = subg.get_edge_data(node1, node2)[cn.REACTION]
    # if reaction.label is not already included in the attribute,
    if reaction.label not in set(reaction_label):
      reaction_label = reaction_label + [reaction.label]
  else:
    reaction_label = [reaction.label]    
  subg.add_edge(node1, node2, reaction=reaction_label)


r58: s36 -> s232
re65: s260 -> s232
re64: s270 -> s232


In [11]:
import tesbml
for error_path in error_paths:
  document = tesbml.readSBML(error_path)
  model = document.getModel()
  #pm.print_model(model)
  try:
    simple.initialize(error_path)
  except:
    print(error_path)
    print("showed error")

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000596_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/MODEL0568648427_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000410_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000081_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000075_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000094_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000353_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000627_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/MODEL0072364382_url.xml
showed error


In [57]:
for reaction in model.getListOfReactions():
  num_rct = len([r.species for r in reaction.getListOfReactants()])
  num_pdt = len([p.species for p in reaction.getListOfProducts()])
  stoi_rct = sum([r.stoichiometry for r in reaction.getListOfReactants()])
  stoi_pdt = sum([p.stoichiometry for p in reaction.getListOfProducts()])
  print("x =", num_rct, ";y =", num_pdt, ";z =", stoi_rct, ";w =", stoi_pdt)
  for reaction_category in REACTION_CATEGORIES:
    if reaction_category.predicate(num_rct, num_pdt, 
                                 stoi_rct, stoi_pdt):
      print(reaction_category.category)

x = 3 ;y = 1 ;z = 4.0 ;w = 2.0
reaction_n_n
x = 3 ;y = 1 ;z = 4.0 ;w = 1.5
reaction_n_n
x = 1 ;y = 1 ;z = 1.0 ;w = 1.0
reaction_1_1
x = 1 ;y = 1 ;z = 1.0 ;w = 1.0
reaction_1_1
